## Miguel Landrito

# Getting data from a News Website using NewsAPI

### Import package

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
head = 'http://newsapi.org/v2/everything?'

### Pick which news source you want to scrape data from
You can do this by setting parameters to filter out a specific source, date, time, language etc. <br>
Through NewsAPI's "everything" url you will be able to select which domain that is supported by them can be retrieved through the requests library <br>
For this homework, I will be using wired.com as my domain for retrieving news articles

In [3]:
# Choose wired.com to be my news site, and filter the articles to be from March 11-12, 2021
params = {
    'domains' : 'wired.com',
    'apiKey' : '54c5ad81c2b748528b5458f21981fe83',
    'from' : '2021-03-11',
    'to' : '2021-03-12',
    'url' : 'wired.com/story'
}

requested = requests.get(head, params = params)
requested_json = requested.json()
requested_json

{'status': 'ok',
 'totalResults': 22,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Brian Barrett',
   'title': "Netflix's Password-Sharing Crackdown Has a Silver Lining",
   'description': "The streaming service is making account owners enter two-factor codes in a limited test. That's… actually not so bad.",
   'url': 'https://www.wired.com/story/netflix-password-sharing-crackdown/',
   'urlToImage': 'https://media.wired.com/photos/604bc784cbf7b34fd7c7e5c3/191:100/w_1280,c_limit/Science_netflix_1300883455.jpg',
   'publishedAt': '2021-03-12T22:53:58Z',
   'content': 'Look, lets be honest. Sharing passwords is as endemic to the Netflix experience as having your favorite show canceled two seasons in. So when the streaming service starts testing ways to curtail that… [+3766 chars]'},
  {'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Eve Sneider',
   'title': 'The Pandemic Turns One, Vaccine Trials Adapt, and More News',
   'description': 'Catch up on

#### Taking the Articles from the JSON

In [4]:
requested_json = requested_json['articles']
requested_json

[{'source': {'id': 'wired', 'name': 'Wired'},
  'author': 'Brian Barrett',
  'title': "Netflix's Password-Sharing Crackdown Has a Silver Lining",
  'description': "The streaming service is making account owners enter two-factor codes in a limited test. That's… actually not so bad.",
  'url': 'https://www.wired.com/story/netflix-password-sharing-crackdown/',
  'urlToImage': 'https://media.wired.com/photos/604bc784cbf7b34fd7c7e5c3/191:100/w_1280,c_limit/Science_netflix_1300883455.jpg',
  'publishedAt': '2021-03-12T22:53:58Z',
  'content': 'Look, lets be honest. Sharing passwords is as endemic to the Netflix experience as having your favorite show canceled two seasons in. So when the streaming service starts testing ways to curtail that… [+3766 chars]'},
 {'source': {'id': 'wired', 'name': 'Wired'},
  'author': 'Eve Sneider',
  'title': 'The Pandemic Turns One, Vaccine Trials Adapt, and More News',
  'description': 'Catch up on the most important updates from this week.',
  'url': 'https:

### CLeaning the JSON

In [5]:
# Sample Code to retrieve the article from the website, since API is limited to only 200 characters

article = requests.get(requested_json[0]['url'])
soup = BeautifulSoup(article.content, 'html.parser')
article_content = soup.find_all("div", {"class": "body__container"})
if not article_content:
    article_content = soup.find_all("article")
article_content_text = article_content[0].get_text()

article_content_text

"Look, let’s be honest.  Sharing passwords is as endemic to the Netflix experience as having your favorite show canceled two seasons in. So when the streaming service starts testing ways to curtail that practice, it understandably riles up the many, many people who have come to expect communal accounts as a matter of course. And yes, it is always annoying when a gravy train goes off the rails. But even if it’s not Netflix’s top priority here, you’re much better off keeping your password to yourself.The limited test that Netflix introduced this week is basically a form of two-factor authentication, the kind you hopefully already have on most of your online accounts. Some users have begun to see the following prompt when settling in for a binge: “If you don’t live with the owner of this account, you need your own account to keep watching.” Below that, there’s an option to get a code emailed or texted to the account owner, which you can enter to continue watching.\xa0A source familiar wit

In [6]:
cleaned = []

for i in range(len(requested_json)):
    article = requests.get(requested_json[i]['url'])
    soup = BeautifulSoup(article.content, 'html.parser')
    article_content = soup.find_all("div", {"class": "body__container"})
    if not article_content:
        article_content = soup.find_all("article")
    article_content_text = article_content[0].get_text()

    articles = {}
    articles['date'] = requested_json[i]['publishedAt']
    articles['title'] = requested_json[i]['title']
    articles['full_article'] = article_content_text
    articles['author'] = requested_json[i]['author']
    
    cleaned.append(articles)
    
    
# for i in range(len(extract_articles)):
#     # Extract full article
#     article = requests.get(extract_articles[i]['url'])
#     soup = BeautifulSoup(article.content, 'html.parser')
#     article_content = soup.find_all("section", {"class": "entry-content"})
#     article_content_text = article_content[0].get_text()
    
#     # Store temporarily in a dictionary
#     results_dict = {}
#     results_dict['date'] = extract_articles[i]['publishedAt']
#     results_dict['title'] = extract_articles[i]['title']
#     results_dict['full_article'] = article_content_text
#     results_dict['author'] = extract_articles[i]['author']
    
#     # Append to list
#     cleaned.append(results_dict)

cleaned

[{'date': '2021-03-12T22:53:58Z',
  'title': "Netflix's Password-Sharing Crackdown Has a Silver Lining",
  'full_article': "Look, let’s be honest.  Sharing passwords is as endemic to the Netflix experience as having your favorite show canceled two seasons in. So when the streaming service starts testing ways to curtail that practice, it understandably riles up the many, many people who have come to expect communal accounts as a matter of course. And yes, it is always annoying when a gravy train goes off the rails. But even if it’s not Netflix’s top priority here, you’re much better off keeping your password to yourself.The limited test that Netflix introduced this week is basically a form of two-factor authentication, the kind you hopefully already have on most of your online accounts. Some users have begun to see the following prompt when settling in for a binge: “If you don’t live with the owner of this account, you need your own account to keep watching.” Below that, there’s an opti

In [7]:
df = pd.DataFrame(cleaned)
df.rename(columns={'date':'Date',
                    'title':'Title',
                    'full_article':'Full Article',
                    'author': 'Author'}, 
                 inplace=True)
df

,Date,Title,Full Article,Author
0,2021-03-12T22:53:58Z,Netflix's Password-Sharing Crackdown Has a Sil...,"Look, let’s be honest. Sharing passwords is a...",Brian Barrett
1,2021-03-12T19:14:48Z,"The Pandemic Turns One, Vaccine Trials Adapt, ...","The pandemic has raged for a year, vaccine tri...",Eve Sneider
2,2021-03-12T18:48:16Z,A Bird-Feed Seller Beat a Chess Master. Then I...,"On March 2, Levy Rozman, in a pink sweater and...",Cecilia D'Anastasio
3,2021-03-12T17:05:00Z,Social Media Reminds Us of the Year That Wasn’t,The Monitor is a weekly column devoted to eve...,Angela Watercutter
4,2021-03-12T17:00:00Z,Tim Wu and Lina Khan Can Now Put Antitrust The...,"Hey, everyone. Nothing like a packed stadium ...",Steven Levy
5,2021-03-12T17:00:00Z,What If the Afterlife Had In-App Purchases?,"\nIn the Amazon original series Upload, a youn...",Geek's Guide to the Galaxy
6,2021-03-12T15:00:00Z,"With Spectre Still Lurking, Google Looks to Pr...",It's been more than three years since research...,Lily Hay Newman
7,2021-03-12T13:17:40Z,"WTF Is an NFT, Anyway? And Should I Care?","When you think of digital media, you probably ...",Wired Staff
8,2021-03-12T13:00:00Z,Ocean Acidification Could Make Tiny Fish Lose ...,An immobilized fish lay between Craig Radford'...,Max G. Levy
9,2021-03-12T13:00:00Z,The Science of Why Your Friends Shot You From ...,It offers neither the narrative escapism of C...,Kyle Hoekstra


In [8]:
with open('articles.json', 'w') as outfile:
    json.dump(cleaned, outfile)